In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
from tqdm.notebook import tqdm
from perlin_noise import PerlinNoise

In [ ]:
a = np.random.rand(1000000)
a.shape

In [ ]:
%%time
b = np.sort(a)

In [ ]:
%%time
c = np.sort(b, kind='mergesort')

In [ ]:
size = (10, 20)

# initialize heigh map with perlin noise
height_map0 = np.zeros(size)
x = np.arange(size[0])
y = np.arange(size[1])
Y, X = np.meshgrid(y, x)

# noise = PerlinNoise(octaves=3, seed=1)
# y_ridge_mean = 20
# ridge_wobble = 0
# wobble_noise = PerlinNoise(octaves=3, seed=2)
# y_ridge = y_ridge_mean + ridge_wobble*np.array([wobble_noise(i/size[0]) for i in range(size[0])])
# y_sigma = 10
# ridge_height = 10

# for i in tqdm(range(size[0])):
#     for j in range(size[1]):
#         height_map0[i,j] = ridge_height/(1 + (j - y_ridge[i])**2 / y_sigma**2) + noise([i/size[0], j/size[1]]) + 1

height_map0 = np.exp(-np.sqrt((X - 0.5*size[0])**2 + (Y-0.5*size[1])**2)/20)


plt.close(1)
fig, ax = plt.subplots(num=1, figsize=(9,6))

plt.imshow(height_map0, cmap='gray')
plt.colorbar()

fig.tight_layout()

In [ ]:
# compute the gradient of the height map
grad = np.gradient(-height_map0)
# height_map0_grad = np.sqrt(height_map0_grad[0]**2 + height_map0_grad[1]**2)


In [ ]:
plt.close(1)
fig, ax = plt.subplots(num=1, figsize=(9,6))

# plt.imshow(height_map0, cmap='gray')
plt.pcolormesh(X,Y,height_map0, cmap='gray')
plt.axis('equal')
plt.colorbar()

plt.quiver(X,Y,grad[0],grad[1], color='red')

fig.tight_layout()